#Fundamentals for Runnable Parallels
Made by: Wilfredo Aaron Sosa Ramos (AI Lab Manager at RealityAI Labs)

Reference: https://python.langchain.com/docs/how_to/parallel/

##1. Install the dependencies

In [4]:
!pip install -q langchain langchain_core langchain_community langchain_google_genai faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 36.6 MB/s eta 0:00:00


In [3]:
import os
from google.colab import userdata

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

##2. Double-Branch and Combination pattern

In [6]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

sentences = [
    "Software architecture defines the high-level structure of a system, including its components and their interactions.",
    "A well-designed software architecture promotes scalability, maintainability, and performance.",
    "Common architectural styles include monolithic, microservices, and event-driven architectures.",
    "Software architects often use diagrams to communicate the design and structure of a system.",
    "Design patterns like MVC (Model-View-Controller) and Repository are commonly employed in software architecture.",
    "Software architecture must balance functional requirements with non-functional requirements like security and reliability.",
    "Decoupling components in a system reduces dependencies and enhances modularity.",
    "Cloud-native architectures leverage containerization and orchestration tools like Kubernetes.",
    "Choosing the right software architecture depends on the project's requirements, timeline, and resource constraints.",
    "Continuous evaluation and evolution of the architecture are crucial to address new challenges and technological advancements."
]

vectorstore = FAISS.from_texts(
    sentences, embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001")
)
retriever = vectorstore.as_retriever()
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

# The prompt expects input with keys for "context" and "question"
prompt = ChatPromptTemplate.from_template(template)

model = ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp")

retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [7]:
retrieval_chain.invoke("What is Software Architecture?")

'Software architecture defines the high-level structure of a system, including its components and their interactions.\n'

In [8]:
retrieval_chain.invoke("How can I create an efficient Software Architecture?")

"The provided documents don't directly explain how to *create* an efficient software architecture, but they do state that a well-designed software architecture promotes scalability, maintainability, and performance. They also mention that choosing the right software architecture depends on the project's requirements, timeline, and resource constraints, and that it must balance functional requirements with non-functional requirements like security and reliability.\n"

##3. Using itemgetter as a shorthand

In [10]:
from operator import itemgetter

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

sentences = [
    "Software architecture defines the high-level structure of a system, including its components and their interactions.",
    "A well-designed software architecture promotes scalability, maintainability, and performance.",
    "Common architectural styles include monolithic, microservices, and event-driven architectures.",
    "Software architects often use diagrams to communicate the design and structure of a system.",
    "Design patterns like MVC (Model-View-Controller) and Repository are commonly employed in software architecture.",
    "Software architecture must balance functional requirements with non-functional requirements like security and reliability.",
    "Decoupling components in a system reduces dependencies and enhances modularity.",
    "Cloud-native architectures leverage containerization and orchestration tools like Kubernetes.",
    "Choosing the right software architecture depends on the project's requirements, timeline, and resource constraints.",
    "Continuous evaluation and evolution of the architecture are crucial to address new challenges and technological advancements."
]

vectorstore = FAISS.from_texts(
    sentences, embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001")
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke({"question": "How can I create an efficient Software Architecture?", "language": "English and Spanish"})

"English:\nBased on the provided context, creating an efficient software architecture involves considering the project's requirements, timeline, and resource constraints. A well-designed architecture will promote scalability, maintainability, and performance. It also needs to balance functional requirements with non-functional requirements like security and reliability.\n\nSpanish:\nSegún el contexto proporcionado, crear una arquitectura de software eficiente implica considerar los requisitos del proyecto, el cronograma y las limitaciones de recursos. Una arquitectura bien diseñada promoverá la escalabilidad, la mantenibilidad y el rendimiento. También debe equilibrar los requisitos funcionales con los no funcionales como la seguridad y la confiabilidad.\n"

##4. Parallelize steps

In [12]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel

quality_attributes_chain = ChatPromptTemplate.from_template("Provide quality attributes for the given software architecture description: {desc}") | model

plan_chain = (
    ChatPromptTemplate.from_template("Plan how to create an efficient software architecture with this description: {desc}") | model
)

map_chain = RunnableParallel(quality_attributes=quality_attributes_chain, plan=plan_chain)

map_chain.invoke({"desc": "Microservices for Commercial LLMs"})

{'quality_attributes': AIMessage(content="Okay, let's break down quality attributes for a microservices architecture designed to support commercial Large Language Models (LLMs). We'll consider various facets of quality that are crucial for this kind of system.\n\n**Understanding the Context**\n\nBefore diving into attributes, it's important to understand the context. A commercial LLM microservices architecture likely needs to handle:\n\n*   **High Traffic:** Potentially many concurrent users.\n*   **Varied Requests:** Different models, prompting styles, input lengths, and potentially different output formats.\n*   **Integration:** Integration with other systems (e.g., authentication, data storage, analytics).\n*   **Scalability:** The ability to grow resources as demand increases.\n*   **Cost Efficiency:** Optimizing resource usage to manage expenses.\n*   **Security:** Protection of sensitive data and models.\n*   **Reliability:** Ensuring the system is available and performs correctl